# Prompt Processing Bean Counting for {{ params.date }}

In [ ]:
date = '2023-12-22'

In [ ]:
dayobs = int(date.replace("-", ""))

In [ ]:
from astropy.time import Time, TimeDelta
import boto3
import json
import pandas
from lsst_efd_client import EfdClient

In [ ]:
import logging
logger = logging.getLogger("analysis")
logger.setLevel(logging.DEBUG)

In [ ]:
async def get_df_from_next_visit_events(date):
    client = EfdClient("usdf_efd")

    start = Time(date, scale="utc", format="isot") + TimeDelta(12*60*60, format="sec")
    end = start + TimeDelta(1, format="jd")
    
    topic = "lsst.sal.ScriptQueue.logevent_nextVisit"
    df = await client.select_time_series(topic, ["*"], start.utc, end.utc)
    all_canceled = await client.select_time_series(topic+"Canceled", ["*"], start.utc, end.utc)

    if df.empty:
        logger.info(f"No events on {date}")
        return None
    
    # Only select on-sky AuxTel imaging survey
    df = df.loc[
        (df["coordinateSystem"] == 2)
        & (df["salIndex"] == 2)
        & (df["survey"] == "AUXTEL_PHOTO_IMAGING")
    ].set_index("groupId")
    logger.info(f"There were {len(df)} AUXTEL_PHOTO_IMAGING nextVisit events on {date}")
    
    # Ignore the explicitly canceled groups
    if not all_canceled.empty:
        canceled = df.index.intersection(all_canceled.set_index("groupId").index).tolist()
        if canceled:
            logger.info(f"{len(canceled)} events were canceled {canceled}")
            df = df.drop(canceled)

    return df

In [ ]:
df_efd = await get_df_from_next_visit_events(date)

In [ ]:
from lsst.daf.butler import Butler
butler = Butler("/repo/embargo", writeable=False)

In [ ]:
results = butler.registry.queryDimensionRecords(
    "exposure",
    datasets="raw",
    collections="LATISS/raw/all",
    where="exposure.science_program IN ('AUXTEL_PHOTO_IMAGING') "
          "and instrument='LATISS' and exposure.day_obs=day_obs  ",
    bind={"day_obs": dayobs},
)
logger.info(f"Found {results.count()} raws in {butler} for {dayobs}")

embargo_records = dict()
for record in results:
    embargo_records[record.id] = record

df_butler = pandas.DataFrame.from_records([embargo_records[num].toDict() for num in embargo_records]).set_index("group_name")

In [ ]:
groups_no_raw = set(df_efd.index) -  set(df_butler.index)
if groups_no_raw:
    logger.info(f"{len(groups_no_raw)} group had records in EFD but no raws in the embargo butler: {groups_no_raw}")

In [ ]:
df_efd = df_efd.drop(groups_no_raw)

In [ ]:
if len(df_efd) != len(df_butler):
    logger.warning("Counts do not match; need attention")

In [ ]:
df_md = pandas.merge(df_efd.reset_index(), df_butler.reset_index(), 
                     how="outer", left_on="groupId", right_on="group_name",
                     suffixes=('_efd', '_butler'),
                     validate="one_to_one",).set_index("groupId")

logger.info(f"Total: {len(df_md)} groups in the table")

In [ ]:
boring_cols = ["instrument_efd", "instrument_butler", "science_program", "observation_reason", "observation_type", 
               "cameraAngle", "has_simulated", "dome", "coordinateSystem", "rotationSystem",
               "private_identity", "private_origin", "private_revCode", "salIndex", "totalCheckpoints",
               "nimages",
               "day_obs", "survey", "exposure_time"]
for col in boring_cols:
    if df_md[col].nunique() == 1:
        logger.debug(f"column {col} has only {df_md[col].unique()}")
        df_md.drop(columns=[col,], inplace=True) 
    else:
        logger.warning(f"Column {col} has {df_md[col].unique()}")

In [ ]:
for col in ("physical_filter", ):
    logger.info(f"Column {col} has {df_md[col].unique()}")

In [ ]:
df_md.columns

In [ ]:
df_md["offset0"] = df_md["position0"] - df_md["tracking_ra"]
df_md["offset1"] = df_md["position1"] - df_md["tracking_dec"]

In [ ]:
df_md[["seq_num", "position0", "tracking_ra", "offset0", "position1", "tracking_dec", "offset1", "physical_filter", "id"]]

In [ ]:
df_md[["offset0", "offset1"]].describe()

In [ ]:
def get_exposure_id(dayobs: int):
    """
    Returns
    -------
    df : `pandas.DataFrame`
    """
    butler = Butler("/repo/embargo", writeable=False)

    results = butler.registry.queryDimensionRecords(
        "exposure",
        datasets="raw",
        collections="LATISS/raw/all",
        where="instrument='LATISS' and exposure.observation_type='science' and"
        " exposure.day_obs=dayobs",
        bind={"dayobs": dayobs},
    )

    df = pandas.DataFrame(
        [(_.group_name, _.id) for _ in results], columns=["groupId", "expId"]
    ).set_index("groupId")

    return df

In [ ]:
def get_df_file(df_expId, butler, datasetType, where="", collections=...):
    """
    Get the last-moditied timestamps of the dataset files in a bucket-based butler repo at USDF

    Returns
    -------
    df : `pandas.DataFrame`
    """ 
    s3_endpoint = "https://s3dfrgw.slac.stanford.edu"
    s3client = boto3.client("s3", endpoint_url=s3_endpoint)
    refs = butler.registry.queryDatasets(
        datasetType=datasetType,
        collections=collections,
        where=where,
    )

    timestamps = dict()
    dimension = None
    for ref in refs:
        if not dimension:
            if "visit" in ref.dataId:
                dimension = "visit"
            else:
                dimension = "exposure"
        fits_uri = butler.getURI(ref)
        time_written = s3client.head_object(
            Bucket=fits_uri.netloc,
            Key=fits_uri.relativeToPathRoot,
        )["LastModified"]
        timestamps[ref.dataId[dimension]] = time_written
        
    df = pandas.DataFrame.from_dict(data=timestamps, orient='index', columns=["file"])  
    df = pandas.merge(df, df_expId.reset_index(), left_index=True, right_on="expId", 
                      how="left", validate="one_to_one",).set_index("groupId")
    return df

In [ ]:
df_expId = get_exposure_id(dayobs)

In [ ]:
df_raw = get_df_file(df_expId, butler, "raw", 
    collections=["LATISS/raw/all"],
    where=f"exposure.science_program IN ('AUXTEL_PHOTO_IMAGING') and instrument='LATISS' and exposure.day_obs={dayobs}")

In [ ]:
if len(df_raw) != len(df_md):
    logger.warning("Counts of raw files do not match; need attention")

In [ ]:
df_md = df_md.merge(df_raw[["file"]], how="outer", left_index=True, right_index=True, validate="one_to_one",)

In [ ]:
df_md.rename(columns={"file": "ts_raw"}, inplace=True)

In [ ]:
df_output = get_df_file(df_expId, butler, "calexp", 
    collections=[f"LATISS/prompt/output-{date}/*/prompt-proto-service-*"])

In [ ]:
df_md = df_md.merge(df_output[["file"]], how="outer", left_index=True, right_index=True, validate="one_to_one",)

In [ ]:
df_md.rename(columns={"file": "ts_output"}, inplace=True)

In [ ]:
!realpath ~

In [ ]:
def get_loki_command_with_phrase(time_start, time_end, phrase, extra=""):
    command = f"""~/binaries/logcli-linux-amd64 --output=jsonl --tls-skip-verify query --addr=http://sdfloki.slac.stanford.edu:80 --timezone=UTC -q --limit=200 --from="{time_start}" --to="{time_end}" --proxy-url=http://sdfproxy.sdf.slac.stanford.edu:3128  '{{namespace="vcluster--usdf-prompt-processing", container="user-container", pod=~"prompt-proto-service-.+"}} |~ "{phrase}" {extra} ' """
    return command

In [ ]:
def get_df_from_loki(date, search_phrase="Waiting for snaps"):
    start = Time(date, scale="utc", format="isot") + TimeDelta(12*60*60, format="sec")
    end = start + TimeDelta(1, format="jd")
    command = get_loki_command_with_phrase(start.strftime('%Y-%m-%dT%H:%M:%SZ'), 
                                           end.strftime('%Y-%m-%dT%H:%M:%SZ'),
                                           search_phrase)
    results = !{command}
    logger.debug(f"Got {len(results)} Loki records for {phrase}")
    data = [json.loads(_) for _ in results]
    df = pandas.json_normalize(data)
    df = df.merge(pandas.json_normalize(df["line"].apply(json.loads)),
                  left_index=True, right_index=True).drop(columns=["line"])

    if "group" not in df.columns and "message" in df.columns:
        df["group"] = df["message"].str.extract(r"groupId='([T:.\d-]*)',")

    df["ts"] = pandas.to_datetime(df["timestamp"])
    
    return df[["group", "ts"]]

In [ ]:
phrases = {
    "unpckMsg": "Unpacked message as \" |~ \"AUXTEL",
    "prepBtlr": "Preparing Butler for visit",
    "waitSnap": "Waiting for snaps",
    "runPipe1": "Running pipeline",
    "pipeSucc": "Pipeline successfully run",
    "outputSa": "Pipeline products saved to collection",
    #"timeout1": "Timed out waiting for image after receiving exposures ",
}

In [ ]:
df_loki = pandas.DataFrame(columns=["group"])
for phrase in phrases:
    df2 = get_df_from_loki(date, phrases[phrase]).rename(columns={"ts": "ts_" + phrase})
    df_loki = df_loki.merge(df2, on="group", how="outer", validate="one_to_one",)
df_loki = df_loki.rename(columns={"group": "groupId"}).set_index("groupId")

In [ ]:
# This plot only uses Loki timestamp.  Hence it includes groups with no data taken.  
df1 = pandas.DataFrame(index=df_loki.index)
ref = "ts_unpckMsg"
for col_name in df_loki.columns:
    # Notes: if the ref column doesn't exist, all become NaN in df1 
    # This can happens e.g. an exposure wasn't taken, so that group isn't in df_md
    df1[col_name] = (df_loki[col_name] - df_loki[ref]).dt.total_seconds()
    
ax = df1.drop(columns=[ref]).plot(kind="hist", title=f"{date}; ref={ref}", xlabel="seconds", ylabel="", bins=50,
         #subplots=True, layout=(2,4), figsize=(12,6),
         alpha=0.5, rot=45, 
)

pandas.merge(df1, df_loki, how="outer", left_index=True, right_index=True, 
             suffixes=('_diff', '_loki'),
             validate="one_to_one",).head()

In [ ]:
df_md["ts_begin"] = df_md["timespan"].apply(lambda _: pandas.Timestamp(_.begin.utc.datetime, tz="UTC"))
df_md["ts_end"] = df_md["timespan"].apply(lambda _: pandas.Timestamp(_.end.utc.datetime, tz="UTC"))

# typically a small fraction of seconds before sndStamp 
df_md["ts_group_utc"] = df_md["group_name"].apply(lambda _: pandas.Timestamp(Time(_, scale="tai").utc.datetime, tz="UTC"))

# time of visit publication; TAI in unix seconds
df_md["ts_sndStamp"] = df_md["private_sndStamp"].apply(lambda _: pandas.Timestamp(Time(_, format="unix_tai").utc.datetime, tz="UTC"))
# time of visit publication; UTC in unix seconds
df_md["ts_efdStamp"] = df_md["private_efdStamp"].apply(lambda _: pandas.Timestamp.fromtimestamp(_, tz="UTC")) 
# Let them be private 
df_md["ts_rcvStamp"] = df_md["private_rcvStamp"].apply(lambda _: pandas.Timestamp.fromtimestamp(_, tz="UTC")) 
df_md["ts_kafkaStamp"] = df_md["private_kafkaStamp"].apply(lambda _: pandas.Timestamp.fromtimestamp(_, tz="UTC")) 

In [ ]:
columns = ["ts_sndStamp", "ts_begin", "ts_end", "ts_raw", "ts_output"]
df0 = pandas.merge(df_md[columns],
                   df_loki, 
                   # only those with data in butler, not those groups with events but no data taken.
                   how="left",
                   left_index=True, right_index=True, validate="one_to_one",)

In [ ]:
df2 = pandas.DataFrame(index=df0.index)
ref = "ts_sndStamp"
for col_name in df0.columns:
    df2[col_name] = (df0[col_name] - df0[ref]).dt.total_seconds()

df2[["ts_sndStamp", "ts_waitSnap","ts_begin","ts_end", "ts_runPipe1", "ts_outputSa", "ts_raw", "ts_output"]].plot(
    kind="hist", title=f"{date}; ref={ref}", xlabel="seconds", ylabel="", bins=150,
    alpha=0.5, rot=45, 
)

df2.describe()

In [ ]:
df2 = pandas.DataFrame(index=df0.index)
ref = "ts_end"
for col_name in df0.columns:
    df2[col_name] = (df0[col_name] - df0[ref]).dt.total_seconds()

df2[["ts_sndStamp", "ts_waitSnap","ts_begin","ts_end", "ts_runPipe1", "ts_raw", "ts_output"]].plot(
    kind="hist", title=f"{date}; ref={ref}", xlabel="seconds", ylabel="", bins=150,
    alpha=0.5, rot=45, 
)

df2.describe()

In [ ]:
ax = df2.plot.box(title=f"{date}; ref={ref}", ylabel="seconds", figsize=(12,5),
             column=df2.median().sort_values().index.tolist(),
)
ax.legend(labels=df2.median().sort_values().to_string().split("\n"), 
          loc="lower right", title="median", handlelength=0)